In [14]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Exercise

Go to http://books.toscrape.com/, Using what you have learned create a csv file the contains all the books found in the website. The csv file should contain the following:

- Title
- Price
- Description
- Availability

Code guides have been provided to help you in creating the web scraper. Found below is the `get_title_links_and_next_page` this function returns 2 things book urls in a page and link to the next page. The idea here is to collect first all the book links available in the website and store the links in the `title_links` variable **(5 points)**

In [15]:
list_links = [] 
    #get the html for the url that was given
page = requests.get('http://books.toscrape.com/index.html')
    #parse the html file for beautifulsoup to query on
soup = BeautifulSoup(page.text, 'html.parser')
    #inspecting the page we notice that the books are placed under 
    #the article tag so we get all articles
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

In [25]:
base_url = "http://books.toscrape.com/"

def get_title_links_and_next_page(page_url):
    #this is where we store our links to the title 
    list_links = [] 
    #get the html for the url that was given
    page = requests.get(page_url)
    #parse the html file for beautifulsoup to query on
    soup = BeautifulSoup(page.text, 'html.parser')
    #inspecting the page we notice that the books are placed under 
    #the article tag so we get all articles
    #print(soup.prettify())

    for article in soup.find_all('article'):
        #the article tag has an anchor tag so we find it and get the href
        if "catalogue" not in article.find("a")['href']:
            url = base_url + "catalogue/" + article.find("a")['href']
        else:
            url = base_url + article.find("a")['href']
        #add the title url to our list of titles 
        list_links.append(url)
    
  
    
    #try to check if a next button is in the page 
    try:
        next_url = soup.find('li', attrs = {'class':'next'}).find("a")['href']
     #   print(soup.find('li', attrs = {'class':'next'}).find("a")['href'])
     #if none we return None :)     
    except:
        next_url = None

    return (list_links, next_url)


#initial set up to crawl the book links and next page
res = get_title_links_and_next_page('http://books.toscrape.com/index.html')
title_links = res[0]

#while we get a next page link keep on crawling for book links
while res[1]:
    #there are cases that the word "catalogue" is not in the link so we add it 
    #so that we can crawl properly
    if "catalogue" not in res[1]:
        page_url = base_url + "catalogue/" + res[1]
    else:
        page_url = base_url + res[1]
    res = get_title_links_and_next_page(page_url)
    title_links += res[0]

title_links

(['http://books.toscrape.com/catalogue/frankenstein_20/index.html', 'http://books.toscrape.com/catalogue/forever-rockers-the-rocker-12_19/index.html', 'http://books.toscrape.com/catalogue/fighting-fate-fighting-6_18/index.html', 'http://books.toscrape.com/catalogue/emma_17/index.html', 'http://books.toscrape.com/catalogue/eat-pray-love_16/index.html', 'http://books.toscrape.com/catalogue/deep-under-walker-security-1_15/index.html', 'http://books.toscrape.com/catalogue/choosing-our-religion-the-spiritual-lives-of-americas-nones_14/index.html', 'http://books.toscrape.com/catalogue/charlie-and-the-chocolate-factory-charlie-bucket-1_13/index.html', 'http://books.toscrape.com/catalogue/charitys-cross-charles-towne-belles-4_12/index.html', 'http://books.toscrape.com/catalogue/bright-lines_11/index.html', 'http://books.toscrape.com/catalogue/bridget-joness-diary-bridget-jones-1_10/index.html', 'http://books.toscrape.com/catalogue/bounty-colorado-mountain-7_9/index.html', 'http://books.toscrap

Once you have a list of all the available book links we loop through the links and use the 4 functions `get_title`, `get_price`, `get_description`, `get_availability` to retrieve the book information. **(10 points)**

In [53]:
def get_title(soup):
    return soup.find('h1').text

def get_price(soup):
    return soup.find('p', attrs={'class':'price_color'}).text
    
def get_description(soup):
    return soup.find('h2').find_next('p').text
    
def get_availability(soup):
    return soup.find('p', attrs={'class':'instock availability'}).text

book_data = []
for title_link in title_links:
    page = requests.get(title_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #print("checking - ", title_link)
    #print("title ", soup.find('h1'))
    #print(soup.prettify())
    
    title = get_title(soup)
    price = get_price(soup)
    description = get_description(soup)
    availability = get_availability(soup)
    
    book_data += [[title, price, description, availability.strip()]]



In [54]:
book_data
df = pd.DataFrame(book_data)
df.columns = ['title', 'price', 'description', 'availability']
display(df.head())

#save to csv file 
df.to_csv("scrape.csv")

print("done saving")

,title,price,description,availability
0,A Light in the Attic,Â£51.77,It's hard to imagine a world without A Light i...,In stock (22 available)
1,Tipping the Velvet,Â£53.74,"""Erotic and absorbing...Written with starling ...",In stock (20 available)
2,Soumission,Â£50.10,"Dans une France assez proche de la nÃ´tre, un ...",In stock (20 available)
3,Sharp Objects,Â£47.82,"WICKED above her hipbone, GIRL across her hear...",In stock (20 available)
4,Sapiens: A Brief History of Humankind,Â£54.23,From a renowned historian comes a groundbreaki...,In stock (20 available)


done saving
